In [240]:
import numpy as np
import pandas as pd

# line_data format: {node 'from', node 'to', r_ij (pu), x_ij (pu), b_shunt_ij (pu), off-nominal tap ratio (pu), line limit (pu)}
line_data = np.array([[1, 2, 0, 0.1, 0, 0, 1.05],
                      [1, 3, 0, 0.25, 0, 0, 1.05],
                      [2, 3, 0, 0.2, 0, 0, 1.05]])

# bus_data format: {node ID, node type(2=PQ,3=PV,1=SLACK), Pl, Ql, Pg, Qg, Vmag, delta}
bus_data = np.array([[1, 1, 0, 0, 0, 0, 1, 0],
                     [2, 2, 0.9, 0.5, 0, 0, 1, 0],
                     [3, 3, 0.7, 0.4, 1.3, 0.1, 1.01, 0]])

In [241]:
line_data = pd.DataFrame(line_data).sort_values(by=[0,1]).values #line data sorting and data type change

bus_data = pd.DataFrame(bus_data).sort_values(by = [0]).values #Bus data sorting and data type change

In [242]:
nline = len(line_data[:, 0])
nbus = int(np.amax(line_data[:, :2]))  # Total Number of buses
base_mva = 100
R = line_data[:, 2]
X = line_data[:, 3]
B = 0.5j * line_data[:, 4]
taps = line_data[:,5]
Bsh = np.zeros(nbus) #1j* bus_data[:, 3]
Z = R + 1j * X
Y = 1 / Z


def tap_handle(x):
    if np.real(x) == 0:
        return 1
    else:
        return 1
    
# Y Bus Formation
def Y_bus():
    Ybus = np.zeros((nbus, nbus), dtype=complex)
    Y1 = np.zeros((nbus, nbus), dtype=complex)
    np.fill_diagonal(Y1,Bsh)
    
    for k in range(nline):
        # Off Diagonal Elements
        Ybus[int(line_data[k, 0]) - 1, int(line_data[k, 1]) - 1] -= (Y[k])/np.conj(tap_handle(taps[k]))
        Ybus[int(line_data[k, 1]) - 1, int(line_data[k, 0]) - 1] -= (Y[k])/tap_handle(taps[k])

        # Diagonal Elements
        Ybus[int(line_data[k, 0]) - 1, int(line_data[k, 0]) - 1] += (Y[k] + B[k])/pow(abs(tap_handle(taps[k])),2)
        Ybus[int(line_data[k, 1]) - 1, int(line_data[k, 1]) - 1] += Y[k] + B[k]
    
    Y_bus_ = Ybus + Y1
    return Y_bus_

In [243]:
Y_bus()

array([[0.-14.j, 0.+10.j, 0. +4.j],
       [0.+10.j, 0.-15.j, 0. +5.j],
       [0. +4.j, 0. +5.j, 0. -9.j]])

In [244]:
b_data = pd.DataFrame()
b_data['Bus_ID'] = bus_data[:, 0]
b_data['Bus_Type'] = bus_data[:, 1]
b_data['PL'] = bus_data[:, 2]
b_data['QL'] = bus_data[:, 3]
b_data['PG'] = bus_data[:, 4]
b_data['QG'] = bus_data[:, 5]
b_data['Vmag'] = bus_data[:, 6]
b_data['Delta'] = bus_data[:, 7] + 0.0001
b_data['Psh'] = -b_data['PG'] + b_data['PL']
b_data['Qsh'] = -b_data['QG'] + b_data['QL']
b_data['Vr'] = b_data['Vmag'] * np.cos(b_data['Delta'])
b_data['Vm'] = b_data['Vmag'] * np.sin(b_data['Delta'])

In [245]:
b_data

,Bus_ID,Bus_Type,PL,QL,PG,QG,Vmag,Delta,Psh,Qsh,Vr,Vm
0,1.0,1.0,0.0,0.0,0.0,0.0,1.00,0.0001,0.0,0.0,1.00,0.000100
1,2.0,2.0,0.9,0.5,0.0,0.0,1.00,0.0001,0.9,0.5,1.00,0.000100
2,3.0,3.0,0.7,0.4,1.3,0.1,1.01,0.0001,-0.6,0.3,1.01,0.000101


In [246]:
n_pv = b_data[b_data['Bus_Type'] == 3].shape[0]
J = np.zeros((nbus*2 + n_pv + 2, nbus*2 + n_pv + 2))
I_hist = np.zeros(nbus*2 + n_pv + 2)
G = np.real(Y_bus())
B = np.imag(Y_bus())

In [247]:
B

array([[-14.,  10.,   4.],
       [ 10., -15.,   5.],
       [  4.,   5.,  -9.]])

In [248]:
#Adding line parameters to Jacobian matrix
for i in range(nbus):
    i = i*2
    for j in range(nbus):
        j = j*2
        if i == j:
            k = int(i/2)
            J[i, i] = G[k,k]
            J[i, i+1] = -B[k, k]
            J[i+1, i] = B[k, k]
            J[i+1, i+1] = G[k, k]
        else:
            k = int(i/2)
            l = int(j/2)
            J[i, j] = -G[k,l]
            J[i, j+1] = B[k, l]
            J[i+1, j] = -B[k, l]
            J[i+1, j+1] = -G[k, l]


In [249]:
def I_r(Vr, Vm, P, Q):
    return (P*Vr + Q*Vm)/(pow(Vr, 2) + pow(Vm, 2))

def I_m(Vr, Vm, P, Q):
    return (P*Vm - Q*Vr)/(pow(Vr, 2) + pow(Vm, 2))

def F_set(Vr, Vm):
    return (pow(Vr, 2) + pow(Vm, 2))

def d_Ir_Vr(Vr, Vm, P, Q):
    V = pow(Vr,2) + pow(Vm,2)
    return (P* (pow(Vm, 2) - pow(Vr, 2)) - 2*Q*Vr*Vm)/pow(V,2)

def d_Ir_Vm(Vr, Vm, P, Q):
    V = pow(Vr,2) + pow(Vm,2)
    return (Q* (pow(Vr, 2) - pow(Vm, 2)) - 2*P*Vr*Vm)/pow(V,2)

def d_Im_Vr(Vr, Vm, P, Q):
    V = pow(Vr,2) + pow(Vm,2)
    return (Q* (pow(Vr, 2) - pow(Vm, 2)) - 2*P*Vr*Vm)/pow(V,2)

def d_Im_Vm(Vr, Vm, P, Q):
    V = pow(Vr,2) + pow(Vm,2)
    return (P* (pow(Vr, 2) - pow(Vm, 2)) - 2*Q*Vr*Vm)/pow(V,2)

def d_Ir_Q(Vr, Vm):
    V = pow(Vr,2) + pow(Vm,2)
    return Vm/V

def d_Im_Q(Vr, Vm):
    V = pow(Vr,2) + pow(Vm,2)
    return -Vr/V

def d_Fset_Vr(Vr):
    return 2*Vr

def d_Fset_Vm(Vm):
    return 2*Vm

def i_hist_pq_r(Vr, Vm, P, Q):
    return d_Ir_Vr(Vr, Vm, P, Q)*Vr + d_Ir_Vm(Vr, Vm, P, Q)*Vm - I_r(Vr, Vm, P, Q)

def i_hist_pq_m(Vr, Vm, P, Q):
    return d_Im_Vr(Vr, Vm, P, Q)*Vr + d_Im_Vm(Vr, Vm, P, Q)*Vm - I_m(Vr, Vm, P, Q)

def i_hist_pv_r(Vr, Vm, P, Q):
    return d_Ir_Vr(Vr, Vm, P, Q)*Vr + d_Ir_Vm(Vr, Vm, P, Q)*Vm + d_Ir_Q(Vr, Vm)*Q - I_r(Vr, Vm, P, Q)

def i_hist_pv_m(Vr, Vm, P, Q):
    return d_Im_Vr(Vr, Vm, P, Q)*Vr + d_Im_Vm(Vr, Vm, P, Q)*Vm + d_Im_Q(Vr, Vm)*Q - I_m(Vr, Vm, P, Q)

def Fset_hist (Vmag, Vr, Vm):
    return Vmag**2 + Vr**2 + Vm**2


In [250]:
x_pq = list()
x_pv = list()
x_slack = list()


def Jacobian(data_df):
    df = data_df.copy(deep = True)
    df['Qsh'] = -df['QG'] + df['QL']

    for b in range(nbus):
        if int(df.loc[b, 'Bus_Type']) == 1:
            J[-2, 0] = 1
            J[-1, 1] = 1
            J[0, -2] = 1
            J[1, -1] = 1

            I_hist[-2] = df.loc[b, 'Vmag']*np.cos(df.loc[b, 'Delta'])
            I_hist[-1] = df.loc[b, 'Vmag']*np.sin(df.loc[b, 'Delta'])

            x_slack.append(b*2)

        elif int(df.loc[b, 'Bus_Type']) == 2:
            b = b*2
            k = int(b/2)
            J[b, b] += d_Ir_Vr(df.loc[k, 'Vr'], df.loc[k, 'Vm'], df.loc[k, 'Psh'], df.loc[k, 'Qsh'])
            J[b, b+1] += d_Ir_Vm(df.loc[k, 'Vr'], df.loc[k, 'Vm'], df.loc[k, 'Psh'], df.loc[k, 'Qsh'])
            J[b+1, b] += d_Im_Vr(df.loc[k, 'Vr'], df.loc[k, 'Vm'], df.loc[k, 'Psh'], df.loc[k, 'Qsh'])
            J[b+1, b+1] += d_Im_Vm(df.loc[k, 'Vr'], df.loc[k, 'Vm'], df.loc[k, 'Psh'], df.loc[k, 'Qsh'])

            I_hist[b] = i_hist_pq_r(df.loc[k, 'Vr'], df.loc[k, 'Vm'], df.loc[k, 'Psh'], df.loc[k, 'Qsh'])
            I_hist[b+1] = i_hist_pq_m(df.loc[k, 'Vr'], df.loc[k, 'Vm'], df.loc[k, 'Psh'], df.loc[k, 'Qsh'])

            x_pq.append(b)
        else:
            b = b*2
            k = int(b/2)
            J[b, b] += d_Ir_Vr(df.loc[k, 'Vr'], df.loc[k, 'Vm'], df.loc[k, 'Psh'], df.loc[k, 'Qsh'])
            J[b, b+1] += d_Ir_Vm(df.loc[k, 'Vr'], df.loc[k, 'Vm'], df.loc[k, 'Psh'], df.loc[k, 'Qsh'])
            J[b, b+2] += d_Ir_Q(df.loc[k, 'Vr'], df.loc[k, 'Vm'])
            J[b+1, b] += d_Im_Vr(df.loc[k, 'Vr'], df.loc[k, 'Vm'], df.loc[k, 'Psh'], df.loc[k, 'Qsh'])
            J[b+1, b+1] += d_Im_Vm(df.loc[k, 'Vr'], df.loc[k, 'Vm'], df.loc[k, 'Psh'], df.loc[k, 'Qsh'])
            J[b+1, b+2] += d_Im_Q(df.loc[k, 'Vr'], df.loc[k, 'Vm'])
            J[b+2, b] += d_Fset_Vr(df.loc[k, 'Vr'])
            J[b+2, b+1] += d_Fset_Vm(df.loc[k, 'Vm'])

            I_hist[b] = i_hist_pv_r(df.loc[k, 'Vr'], df.loc[k, 'Vm'], df.loc[k, 'Psh'], df.loc[k, 'Qsh'])
            I_hist[b+1] = i_hist_pv_m(df.loc[k, 'Vr'], df.loc[k, 'Vm'], df.loc[k, 'Psh'], df.loc[k, 'Qsh'])
            I_hist[b+2] = Fset_hist(df.loc[k, 'Vmag'],df.loc[k, 'Vr'], df.loc[k, 'Vm'])
            x_pv.append(b)
    
    return I_hist, J
            

In [251]:
def N_R():
    df = b_data.copy(deep=True)
    x_hist = np.zeros(nbus*2 + n_pv + 2)
    x_hist[0] = df.loc[0, 'Vr'] 
    x_hist[1] = df.loc[0, 'Vm']
    x_hist[2] = df.loc[1, 'Vr'] 
    x_hist[3] = df.loc[1, 'Vm'] 
    x_hist[4] = df.loc[2, 'Vr'] 
    x_hist[5] = df.loc[2, 'Vm'] 
    x_hist[6] = df.loc[2, 'Qsh']
    x_hist[7] = 1
    x_hist[8] = 0
    for i in range(1000):
        I_hist, J = Jacobian(df)
        print("------------------------------------------")
        print(I_hist)
        print("------------------------------------------")
        print(J)
        print("------------------------------------------")
    
        X = np.linalg.solve(J, I_hist)
        print("--------- ",i , np.linalg.norm(X - x_hist))
        print(pd.DataFrame(x_hist.T, X.T))
        if np.linalg.norm(X - x_hist) <= 0.0001:
            return df
        
        #df.loc[0, 'Vr'] +=X[0]
        #df.loc[0, 'Vm'] +=X[1]    
        df.loc[1, 'Vr'] +=X[2]
        df.loc[1, 'Vm'] +=X[3]
        df.loc[2, 'Vr'] +=X[4]
        df.loc[2, 'Vm'] +=X[5]
        df.loc[2, 'QG'] +=X[6]
        
        x_hist = X


In [252]:
df_ = N_R()

------------------------------------------
[ 0.00000000e+00  0.00000000e+00 -1.80009999e+00  9.99819975e-01
  1.18808910e+00  2.97148501e-01  2.04020000e+00  9.99999995e-01
  9.99999998e-05]
------------------------------------------
[[ 0.00000000e+00  1.40000000e+01 -0.00000000e+00  1.00000000e+01
  -0.00000000e+00  4.00000000e+00  0.00000000e+00  1.00000000e+00
   0.00000000e+00]
 [-1.40000000e+01  0.00000000e+00 -1.00000000e+01 -0.00000000e+00
  -4.00000000e+00 -0.00000000e+00  0.00000000e+00  0.00000000e+00
   1.00000000e+00]
 [-0.00000000e+00  1.00000000e+01 -9.00099982e-01  1.54998200e+01
  -0.00000000e+00  5.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [-1.00000000e+01 -0.00000000e+00 -1.45001800e+01  8.99899982e-01
  -5.00000000e+00 -0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [-0.00000000e+00  4.00000000e+00 -0.00000000e+00  5.00000000e+00
   5.88118800e-01  9.29420644e+00  9.90099008e-05  0.00000000e+00
   0.00000000e+00]
 [-4.000000

In [253]:
df_

In [254]:
df_['Vmag'] = np.sqrt(df_['Vr']**2 + df_['Vm']**2)
df_['Delta'] = np.arctan(df_['Vm']/df_['Vr'])

TypeError: 'NoneType' object is not subscriptable

In [ ]:
df_

,Bus_ID,Bus_Type,PL,QL,PG,QG,Vmag,Delta,Psh,Qsh,Vr,Vm
0,1.0,1.0,0.0,0.0,0.0,0.0,1.000000,0.000100,0.0,0.000000,1.000000,0.000100
1,2.0,2.0,0.9,0.5,0.0,0.0,382.766727,-1.523949,0.9,0.500000,17.925132,-382.346776
2,3.0,3.0,0.7,0.4,1.3,0.1,270.417964,0.302797,-0.6,-2828.020287,258.115618,80.636240


In [ ]:
X[x_slack[0]: x_slack[0]+2]

array([0.1 , 0.25])

In [ ]:
X

array([0.1 , 0.25, 0.2 ])